In [1]:
from pynq import Overlay
from math import log
import time
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pynq import allocate

In [2]:
ol = Overlay("reconfig_fft.bit")

In [3]:
# ol?

In [4]:
data_dma = ol.fft_block.data_dma
config_dma = ol.fft_block.config_dma

In [5]:
data_send = data_dma.sendchannel
data_recv = data_dma.recvchannel
conf_send = config_dma.sendchannel

# Create Configuration

PAD 00000
FWD/INV 001
PAD 0
CP_LEN 0000000
PAD 000
NFFT 01010

In [6]:
# Pad all values till they are multiples of 8 bits
# Last 5 bits select FFT size (for 1024 or 2^10 point fft we need to set the value to 10)
# We can set the values for the second last byte as all 0
# MSByte sets the IP as FFT or IFFT. We set its value to 1 as we only have 1 channel.

def convert_to_data(fft_direction, size):
    fft_direction.zfill(8)       # 011 -> 00000011
    byte2 = '0'*8

    x = int(log(size, 2))
    fft_size = bin(x)[2:]    # Converts the integer x to its binary representation and strips off the '0b' prefix

    fft_size.zfill(8)

    tdata = fft_direction + byte2 + fft_size

    return int(tdata,2)     # Converts the binary string tdata to an integer using base 2

# int is a 4 byte (32 bit) value

# Create Data

In [7]:
f1 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f1", value=10)
f2 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f2", value=30)
f3 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f3", value=50)
A1 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A1", value=100)
A2 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A2", value=150)
A3 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A3", value=200)
c1 = widgets.Checkbox(value=True, description="PS Implementation")
c2 = widgets.Checkbox(value=True, description="PL Implementation")
samples = widgets.Dropdown(options=[64, 128, 256, 512, 1024, 2048, 4096, 8192], value=8192, description='samples')
T = 1

In [8]:
def create_data(A1, A2, A3, f1, f2, f3, samples):
    t = np.linspace(0, T, samples)
    data = A1*np.sin(2*np.pi*f1*t) + A2*np.sin(2*np.pi*f2*t) + A3*np.sin(2*np.pi*f3*t)
    return data, t

def plot_graph(data, t):
    plt.plot(t, np.abs(data), label = "Input Data")
    plt.xlabel('Time (s)')
    plt.ylabel('Absolute Value')
    plt.legend()
    plt.show()

# FFT function with options for PS PL Implementation

In [9]:
def plot_fft(A1, A2, A3, f1, f2, f3, samples, ps, pl):
    data, t = create_data(A1, A2, A3, f1, f2, f3, samples)
    plot_graph(data, t)
    freq = np.fft.fftshift(np.fft.fftfreq(samples, T/samples))
    
    if ps:
        st = time.time()
        ps_output = np.abs(np.fft.fft(data, samples)) / samples
        et = time.time()
        
        print("PS Execution Time: " + str(et - st))
        
        ps_output_shifted = np.fft.fftshift(ps_output)
        plt.plot(freq, ps_output_shifted, label='PS Output')
        
    if pl:
        conf_buffer = allocate(1, np.uint32)
        conf_buffer[0] = convert_to_data('1', samples)
        
        input_buffer = allocate(samples, np.csingle)
        output_buffer = allocate(samples, np.csingle)
        np.copyto(input_buffer, data)
        
        st = time.time()
        conf_send.transfer(conf_buffer)
        conf_send.wait()
        
        data_send.transfer(input_buffer)
        data_recv.transfer(output_buffer)
        data_send.wait()
        data_recv.wait()
        et = time.time()
        
        print("PL Execution Time: " + str(et - st))
        
        pl_output_shifted = np.fft.fftshift(np.abs(output_buffer) / samples)
        plt.plot(freq, pl_output_shifted, label='PL Output')
        
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.legend()
    plt.show()
        
    if ps and pl:
        plt.figure()
        plt.plot(freq, np.abs(ps_output_shifted - pl_output_shifted), label='Difference')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude')
        plt.legend()
        plt.show()

In [10]:
widgets.interact_manual(plot_fft, A1=A1, A2=A2, A3=A3, f1=f1, f2=f2, f3=f3, samples=samples, ps=c1, pl=c2)

interactive(children=(FloatSlider(value=100.0, description='A1', max=200.0, min=100.0, step=1.0), FloatSlider(…

<function __main__.plot_fft>